In [1]:
import math
from enum import Enum

class CellType(Enum):
    clay = 0
    flowing = 1
    water = 2
    spring = 3
    
    def blocking(self):
        return self == CellType.clay or self == CellType.water

class Underground:
    def __init__(self, cells):
        self.cells = cells
        self.y_min = min(y for (x, y), c in self.cells.items() if c == CellType.clay)
        self.y_max = max(y for (x, y), c in self.cells.items() if c == CellType.clay)
    
    def borders(self):
        x_min = min(x for x, y in self.cells.keys())
        x_max = max(x for x, y in self.cells.keys())
        return x_min, x_max, 0, self.y_max

    def blocking(self, x, y):
        return (x, y) in self.cells and self.cells[(x, y)].blocking()
    
    def __str__(self):
        x_min, x_max, y_min, y_max = self.borders()
        string = ""
        for y in range(y_min, y_max + 1):
            string += "{:04d} ".format(y)
            for x in range(x_min, x_max + 1):
                cell = self.cells.get((x, y))
                if not cell:
                    string += '.'
                elif cell == CellType.clay:
                    string += '#'
                elif cell == CellType.flowing:
                    string += '|'
                elif cell == CellType.water:
                    string += '~'
                else:
                    string += '+'
            string += "\n"
        return string
    
    def flow(self, x = 500, y = 0):
        while y < self.y_max and not (x, y + 1) in self.cells:
            y += 1
            self.cells[(x, y)] = CellType.flowing
        if not self.blocking(x, y + 1):
            return
        new_sources = self.spread(x, y)
        for x_new, y_new in new_sources:
            self.flow(x_new, y_new)
    
    def spread(self, x, y):
        self.cells[(x, y)] = CellType.flowing
        
        new_sources = []
        
        # Spread to the left
        x1 = x
        while self.blocking(x1, y + 1) and not self.blocking(x1 - 1, y):
            x1 -= 1
            self.cells[(x1, y)] = CellType.flowing
        if not self.blocking(x1, y + 1):
            # We reached the end of a ledge, so we start flowing from here
            new_sources.append((x1, y))
        
        # Spread to the right
        x2 = x
        while self.blocking(x2, y + 1) and not self.blocking(x2 + 1, y):
            x2 += 1
            self.cells[(x2, y)] = CellType.flowing
        if not self.blocking(x2, y + 1):
            # We reached the end of a ledge, so we start flowing from here
            new_sources.append((x2, y))
        
        # If we reached a ledge, we will flow again
        if new_sources:
            return new_sources
        # If we have reached no ledge, it means we are in a reservoir!
        # We replace all the line of flowing water with stagnant water
        for x_water in range(x1, x2 + 1):
            self.cells[(x_water, y)] = CellType.water
        # And we start another layer of spreading on top of it
        return self.spread(x, y - 1)
    

In [2]:
import re

# Input
parser_x = re.compile(r"x=(\d+), y=(\d+)..(\d+)")
parser_y = re.compile(r"y=(\d+), x=(\d+)..(\d+)")

cells = {(500, 0): CellType.spring}
with open("Input/17.txt") as file:
    for line in file:
        match = parser_x.match(line)
        if match:
            x, y1, y2 = [int(x) for x in match.groups()]
            for y in range(y1, y2 + 1):
                cells[(x, y)] = CellType.clay
            continue
        match = parser_y.match(line)
        if match:
            y, x1, x2 = [int(x) for x in match.groups()]
            for x in range(x1, x2 + 1):
                cells[(x, y)] = CellType.clay
            continue            
    underground = Underground(cells)

underground.flow()

In [3]:
print("Part 1: {}".format(len([c for (x, y), c in underground.cells.items() if (c == CellType.water or c == CellType.flowing) and (y >= underground.y_min and y <= underground.y_max)])))

Part 1: 28246


In [4]:
print("Part 2: {}".format(len([c for (x, y), c in underground.cells.items() if c == CellType.water and (y >= underground.y_min and y <= underground.y_max)])))

Part 2: 23107
